In [28]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch import optim
from torch import nn

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms

import torchvision.models as models

import random
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image

def seed_everything(seed): # seed 고정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    
def extract_day(images):
    day = int(images.split('.')[-2][-2:])
    return day

def make_day_array(images):
    day_array = np.array([extract_day(x) for x in images])
    return day_array

def make_combination(length, species, data_frame, direct_name):
    before_file_path = []
    after_file_path = []
    time_delta = []

    for i in range(length):
        
        direct = random.randrange(0,len(direct_name))
        temp = data_frame[data_frame['version'] == direct_name[direct]]
    
        sample = temp[temp['species'] == species].sample(2)
        after = sample[sample['day'] == max(sample['day'])].reset_index(drop=True)
        before = sample[sample['day'] == min(sample['day'])].reset_index(drop=True)

        before_file_path.append(before.iloc[0]['file_name'])
        after_file_path.append(after.iloc[0]['file_name'])
        delta = int(after.iloc[0]['day'] - before.iloc[0]['day'])
        time_delta.append(delta)

    combination_df = pd.DataFrame({
        'before_file_path': before_file_path,
        'after_file_path': after_file_path,
        'time_delta': time_delta,
    })

    combination_df['species'] = species

    return combination_df



## 학습 데이터 만들기

베이스라인 코드 덧글에서 'qwopqwop' 님과 '네네넹' 님이 논의 하신 것과 같이 기존의 코드를 돌리면 **병목 현상**이 발생되어서 학습에 많은 어려움이 있었습니다. 이러한 부분을 없애고자 사전에 모든 이미지에 대한 기본적인 전처리를 수행했고 이를 .np 형으로 저장을 해서 가져오는 방식을 택했습니다.

또한 기존의 Baseline 코드와 다른점이라고 한다면 랜덤한 경우의 수를 가져오는 것이 아닌   
**동일 품종 동일 식물(같은 폴더)**


를 가져온다는 것 입니다.

In [29]:
seed_everything(2048)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [30]:
root_path = './drive/MyDrive/Colab Notebooks/224size_train'

# BC 폴더와 LT 폴더에 있는 하위 폴더를 저장한다.
bc_direct = glob(root_path + '/BC/*')
bc_direct_name = [x[-5:] for x in bc_direct]
lt_direct = glob(root_path + '/LT/*')
lt_direct_name = [x[-5:] for x in lt_direct]

# 하위 폴더에 있는 이미지들을 하위 폴더 이름과 매칭시켜서 저장한다.
bc_images = {key : glob(name + '/*.png') for key,name in zip(bc_direct_name, bc_direct)}
lt_images = {key : glob(name + '/*.png') for key,name in zip(lt_direct_name, lt_direct)}

# 하위 폴더에 있는 이미지들에서 날짜 정보만 따로 저장한다.
bc_dayes = {key : make_day_array(bc_images[key]) for key in bc_direct_name}
lt_dayes = {key : make_day_array(lt_images[key]) for key in lt_direct_name}

bc_dfs = []
for i in bc_direct_name:
    bc_df = pd.DataFrame({
        'file_name':bc_images[i],
        'day':bc_dayes[i],
        'species':'bc',
        'version':i
    })
    bc_dfs.append(bc_df)
    
lt_dfs = []
for i in lt_direct_name:
    lt_df = pd.DataFrame({
        'file_name':lt_images[i],
        'day':lt_dayes[i],
        'species':'lt',
        'version':i
    })
    lt_dfs.append(lt_df)

bc_dataframe = pd.concat(bc_dfs).reset_index(drop=True)
lt_dataframe = pd.concat(lt_dfs).reset_index(drop=True)
total_dataframe = pd.concat([bc_dataframe, lt_dataframe]).reset_index(drop=True)


bc_combination = make_combination(5000, 'bc', total_dataframe, bc_direct_name)
lt_combination = make_combination(5000, 'lt', total_dataframe, lt_direct_name)

bc_train = bc_combination.iloc[:4500]
bc_valid = bc_combination.iloc[4500:]

lt_train = lt_combination.iloc[:4500]
lt_valid = lt_combination.iloc[4500:]

train_set = pd.concat([bc_train, lt_train])
valid_set = pd.concat([bc_valid, lt_valid])

In [ ]:
!pip install timm

     |████████████████████████████████| 376 kB 6.7 MB/s 


In [31]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn

train_img_file_names = zip(train_set['before_file_path'], train_set['after_file_path'])
val_img_file_names = zip(valid_set['before_file_path'], valid_set['after_file_path'])

train_before = []
train_after = []
val_before = []
val_after = []

transform = transforms.Compose([
    transforms.ToTensor()
])

for before, after in train_img_file_names:
    before_image = Image.open(before)
    after_image = Image.open(after)

    before_image = transform(before_image)
    after_image = transform(after_image)

    train_before.append(before_image)
    train_after.append(after_image)


for before, after in val_img_file_names:
    before_image = Image.open(before)
    after_image = Image.open(after)

    # validation transform x
    val_before.append(before_image)
    val_after.append(after_image)


torch_train_before = np.zeros((4500,3,224, 224))
torch_train_after = np.zeros((4500,3,224, 224))


for i in range(4500):
    torch_train_before[i] = train_before[i].numpy()
    torch_train_after[i] = train_after[i].numpy()

np.save("train_before.npy", torch_train_before)
np.save("train_after.npy", torch_train_after)
np.save("train_time_delta.npy", np.array(train_set['time_delta']))

In [32]:
class KistDataset(Dataset):
    def __init__(self, combination_df, is_valid= None, is_test= None):

        self.combination_df = combination_df
        self.is_valid = is_valid
        self.is_test = is_test
        if is_valid == None and is_test == None:
            self.transform = transforms.Compose([                                    
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomAffine((-20, 20)),
            transforms.RandomRotation(90),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transforms.Compose([
            transforms.ToTensor(),                                     
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __getitem__(self, idx):
        before_image = Image.open(self.combination_df.iloc[idx]['before_file_path'])
        after_image = Image.open(self.combination_df.iloc[idx]['after_file_path'])

        before_image = self.transform(before_image)
        after_image = self.transform(after_image)
        if self.is_test:
            return before_image, after_image
        time_delta = self.combination_df.iloc[idx]['time_delta']
        return before_image, after_image, time_delta

    def __len__(self):
        return len(self.combination_df)

In [33]:
class CompareCNN(nn.Module):

    def __init__(self):
        super(CompareCNN, self).__init__()
        self.regnet = model = timm.create_model('regnetx_004', pretrained=True, num_classes=1)

    def forward(self, input):
        x = self.regnet(input)
        return x


class CompareNet(nn.Module):

    def __init__(self):
        super(CompareNet, self).__init__()
        self.before_net = CompareCNN()
        self.after_net = CompareCNN()

    def forward(self, before_input, after_input):
        before = self.before_net(before_input)
        after = self.after_net(after_input)
        delta = after - before
        return delta

In [35]:
import numpy as np
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random

            
lr = 1e-5
epochs = 10
batch_size = 64
valid_batch_size = 50

model = CompareNet().to(device)

train_dataset = KistDataset(train_set)
valid_dataset = KistDataset(valid_set)

optimizer = optim.Adam(model.parameters(), lr=lr)

train_data_loader = DataLoader(train_dataset,
                               batch_size=batch_size,
                               shuffle=True)

valid_data_loader = DataLoader(valid_dataset,
                               batch_size=valid_batch_size)

In [36]:
for epoch in tqdm(range(epochs)):
    for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):
        before_image = before_image.to(device)
        after_image = after_image.to(device)
        time_delta = time_delta.to(device)

        optimizer.zero_grad()
        logit = model(before_image, after_image)
        train_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) /
                      torch.LongTensor([batch_size]).squeeze(0).to(device))
        train_loss.backward()
        optimizer.step()

        if step % 15 == 0:
            print('\n=====================loss=======================')
            print(f'\n=====================EPOCH: {epoch}=======================')
            print(f'\n=====================step: {step}=======================')
            print('MAE_loss : ', train_loss.detach().cpu().numpy())

    valid_losses = []
    with torch.no_grad():
        for valid_before, valid_after, time_delta in tqdm(valid_data_loader):
            valid_before = valid_before.to(device)
            valid_after = valid_after.to(device)
            valid_time_delta = time_delta.to(device)


            logit = model(valid_before, valid_after)
            valid_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - valid_time_delta.float())) /
                          torch.LongTensor([valid_batch_size]).squeeze(0).to(device))
            valid_losses.append(valid_loss.detach().cpu())


    print(f'VALIDATION_LOSS MAE : {sum(valid_losses)/len(valid_losses)}')
    checkpoiont = {
        'model': model.state_dict(),

    }

    torch.save(checkpoiont, './drive/MyDrive/Colab Notebooks/regnet_x_004_v2.pt')

  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]


=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 0=======================
MAE_loss :  13.267643

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 15=======================
MAE_loss :  14.647044

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 30=======================
MAE_loss :  14.3661785

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 45=======================
MAE_loss :  12.572607

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 60=======================
MAE_loss :  15.758207

=====================loss=======================

=====================EPOCH: 0====================

  0%|          | 0/20 [00:00<?, ?it/s]

VALIDATION_LOSS MAE : 11.970113754272461


...

VALIDATION_LOSS MAE : 3.033789873123169


0it [00:00, ?it/s]


=====================loss=======================

=====================EPOCH: 9=======================

=====================step: 0=======================
MAE_loss :  2.9327145

=====================loss=======================

=====================EPOCH: 9=======================

=====================step: 15=======================
MAE_loss :  3.208847

=====================loss=======================

=====================EPOCH: 9=======================

=====================step: 30=======================
MAE_loss :  2.1240866

=====================loss=======================

=====================EPOCH: 9=======================

=====================step: 45=======================
MAE_loss :  2.3902044

=====================loss=======================

=====================EPOCH: 9=======================

=====================step: 60=======================
MAE_loss :  2.5701613

=====================loss=======================

=====================EPOCH: 9======================

  0%|          | 0/20 [00:00<?, ?it/s]

VALIDATION_LOSS MAE : 2.630824327468872


In [38]:
test_set = pd.read_csv('./drive/MyDrive/Colab Notebooks/224size_test/test_data.csv')
test_set['l_root'] = test_set['before_file_path'].map(lambda x: './drive/MyDrive/Colab Notebooks/224size_test/' + x.split('_')[1] + '/' + x.split('_')[2])
test_set['r_root'] = test_set['after_file_path'].map(lambda x: './drive/MyDrive/Colab Notebooks/224size_test/' + x.split('_')[1] + '/' + x.split('_')[2])
test_set['before_file_path'] = test_set['l_root'] + '/' + test_set['before_file_path'] + '.png'
test_set['after_file_path'] = test_set['r_root'] + '/' + test_set['after_file_path'] + '.png'


test_dataset = KistDataset(test_set, is_test=True)
test_data_loader = DataLoader(test_dataset,
                               batch_size=64)

test_set

,idx,before_file_path,after_file_path,l_root,r_root
0,0,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...
1,1,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...
2,2,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...
3,3,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...
4,4,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...
...,...,...,...,...,...
3955,3955,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...
3956,3956,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...,./drive/MyDrive/Colab Notebooks/224size_test/L...
3957,3957,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...
3958,3958,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...,./drive/MyDrive/Colab Notebooks/224size_test/B...


In [39]:
test_value = []
with torch.no_grad():
    for test_before, test_after in tqdm(test_data_loader):
        test_before = test_before.to(device)
        test_after = test_after.to(device)
        logit = model(test_before, test_after)
        value = logit.squeeze(1).detach().cpu().float()
        
        test_value.extend(value)

  0%|          | 0/62 [00:00<?, ?it/s]

In [40]:
# 모델 불러오기
model = CompareNet() # 모델 초기화

#evice = torch.device('cpu')
checkpoint = torch.load('./drive/MyDrive/Colab Notebooks/regnet_x_004_v2.pt', map_location=device)
model.load_state_dict(checkpoint['model'])

model.eval() # 드롭아웃 및 배치 정규화를 평가

CompareNet(
  (before_net): CompareCNN(
    (regnet): RegNet(
      (stem): ConvBnAct(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (act): ReLU(inplace=True)
        )
      )
      (s1): RegStage(
        (b1): Bottleneck(
          (conv1): ConvBnAct(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNormAct2d(
              32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (act): ReLU(inplace=True)
            )
          )
          (conv2): ConvBnAct(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2, bias=False)
            (bn): BatchNormAct2d(
              32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (act): ReLU(inplace=True)
            )
          )

In [41]:
# submission 형식을 불러온다.
submission = pd.read_csv('./drive/MyDrive/Colab Notebooks/sample_submission.csv')

# 예측한 값들은 텐서 형태로 변환 시켜준다.
predict = torch.FloatTensor(test_value)

# 음수의 값을 갖는 모든 값들을 1 Day 차이가 발생하도록 바꿔줌
temp_predict = predict.numpy()
temp_predict[np.where(temp_predict<1)] = 1

In [42]:
# 모델의 예측 값을 저장함
submission['time_delta'] = temp_predict
submission.to_csv('./drive/MyDrive/Colab Notebooks/regnetx_004_v2.csv', index=False)